In [149]:
#Data preprocessing important libraries
import pandas as pd  #Data manipulation
import string  #Remove punctuation & characters
import nltk  #Natural language processing 

from nltk.corpus import stopwords  #Stop word removal
from nltk.tokenize import word_tokenize  #Tokenizition
from nltk.stem import PorterStemmer  #Stemming

In [151]:
#Read excel file
text = pd.read_excel(r'C:\Users\HUAWEI\Downloads\Telegram Desktop\classfication of BBC news.xlsx')

In [153]:
#Remove unwanted characters
text['text'] = text['text'].str.replace(f'[{string.punctuation}]', '', regex=True)

In [155]:
#Normalization- convert text column into lower case 
text['text'] = text['text'].str.lower()

In [166]:
#Tokenization
def tokenize_text(text):
    return nltk.word_tokenize(text)

#Apply the tokenization function to the text column
text['tokens'] = text['text'].apply(tokenize_text)

In [167]:
#Define stop words for English
stop_words = set(stopwords.words('english'))

#Stop word removal
def remove_stop_words(tokens):
    return [word for word in tokens if word not in stop_words]

#Apply the function to the tokens column
text['tokens'] = text['tokens'].apply(remove_stop_words)

In [162]:
#Stemming
#Initialize the Porter Stemmer
stemmer = PorterStemmer()

#Function to stem words
def stem_words(text):
    return ' '.join(stemmer.stem(word) for word in nltk.word_tokenize(text))

# Apply the stemming function to the text column
text['text'] = text['text'].apply(stem_words)

In [163]:
#save the changes
text.to_excel('C:\\Users\\HUAWEI\\Downloads\\Telegram Desktop\\classfication of BBC news.xlsx', index=False)